In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('tips.csv')
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251


Метод apply() позволяет применить функцию ко всем выбранным строкам

Пример: нужно получить последний 4 цифры номеров карт

In [ ]:
def last_four(num):
  return str(num)[-4:]

In [ ]:
df['last_four'] = df['CC Number'].apply(last_four)

In [ ]:
df['last_four'].head()

0    3410
1    9230
2    1322
3    5994
4    7221
Name: last_four, dtype: object

Создали свою функцию и применили ее ко всем строкам по столбику CC Number

In [ ]:
df['total_bill'].mean()

19.78594262295082

Пример: в зависимости от суммы чека дать рейтинг ресторану(дешевый,средний или дорогой)

In [ ]:
def yelp(price):

  if price < 10:
    return '$'
  elif price >= 10 and price < 30:
    return '$$'
  else:
    return '$$$'

In [ ]:
df['yelp'] = df['total_bill'].apply(yelp)

In [ ]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,yelp
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322,$$
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,5994,$$
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,7221,$$


Созданные функции должны принимать только одно значение и возвращать одно значение. Также функция не должна возвращать объект Series, так как Pandas его создаст при примении функции к строкам

Теперь рассмотрим как передать в функцию значения двух и более столбцов

In [ ]:
df['total_bill'].apply(lambda num : num * 2) #делается это с помощью lambda функции, вот пример как ее использовать

0      33.98
1      20.68
2      42.02
3      47.36
4      49.18
       ...  
239    58.06
240    54.36
241    45.34
242    35.64
243    37.56
Name: total_bill, Length: 244, dtype: float64

In [ ]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID,last_four,yelp
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959,3410,$$
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608,9230,$$
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458,1322,$$
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260,5994,$$
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251,7221,$$


In [ ]:
def quality(total_bill, tip): #аргументы лучше называть именами столбцов, так будет понятнее при чтении кода
  if tip/total_bill > 0.25:
    return 'Щедрые чаевые'
  else:
    return 'Обычные чаевые'

In [ ]:
df['quality'] = df[['total_bill', 'tip']].apply(lambda df: quality(df['total_bill'], df['tip']), axis = 1)

Берем список из двух столбцов, вызываем метод apply(), в нем пишем lambda выражение, в котором на вход подается DataFrame, а на выходе результат функции quality, в методе apply() указываем второй аргумент, axis = 1(для столбцов) для корректной работы. Можно писать конструкции для трех и более параметров.

Ускорение с помощью np.vectorize()

In [ ]:
df['quality'] = np.vectorize(quality)(df['total_bill'], df['tip']) #2 аргмента в разных скобках. 1)функция 2)аргументы

Преобразовали обычную функцию в функцию, которая работает в режиме broadcast

Тест скорости двух функций для применения метода apply с несколькими параметрами

In [ ]:
import timeit

# этот фрагмент кода будет запускаться только один раз
setup = '''
import numpy as np
import pandas as pd
df = pd.read_csv('tips.csv')
def quality(total_bill,tip):
    if tip/total_bill  > 0.25:
        return "Generous"
    else:
        return "Other"
'''

# это фрагменты кода, для которых будет измеряться время выполнения
# (они будут выполняться много раз)
stmt_one = '''
df['Tip Quality'] = df[['total_bill','tip']].apply(lambda df: quality(df['total_bill'],df['tip']),axis=1)
'''

stmt_two = '''
df['Tip Quality'] = np.vectorize(quality)(df['total_bill'], df['tip'])
'''

In [ ]:
timeit.timeit(setup = setup,
                    stmt = stmt_one,
                    number = 1000)

2.4718158569999957

In [ ]:
timeit.timeit(setup = setup,
                    stmt = stmt_two,
                    number = 1000)

0.1967878209999867

Как итог, видим, что np.vectorize() быстрее

Описание данных и сортировка

In [ ]:
df = pd.read_csv('tips.csv')

In [ ]:
df.describe() #вывели основные характеристики числовых данных

,total_bill,tip,size,price_per_person,CC Number
count,244.000000,244.000000,244.000000,244.000000,2.440000e+02
mean,19.785943,2.998279,2.569672,7.888197,2.563496e+15
std,8.902412,1.383638,0.951100,2.914234,2.369340e+15
min,3.070000,1.000000,1.000000,2.880000,6.040679e+10
25%,13.347500,2.000000,2.000000,5.800000,3.040731e+13
50%,17.795000,2.900000,2.000000,7.255000,3.525318e+15
75%,24.127500,3.562500,3.000000,9.390000,4.553675e+15
max,50.810000,10.000000,6.000000,20.270000,6.596454e+15


In [ ]:
df.sort_values('tip') #сортировка данных по колонке tips в порядке возрастания

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
92,5.75,1.00,Female,Yes,Fri,Dinner,2,2.88,Leah Ramirez,3508911676966392,Fri3780
111,7.25,1.00,Female,No,Sat,Dinner,1,7.25,Terri Jones,3559221007826887,Sat4801
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
...,...,...,...,...,...,...,...,...,...,...,...
141,34.30,6.70,Male,No,Thur,Lunch,6,5.72,Steven Carlson,3526515703718508,Thur1025
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590


In [ ]:
df.sort_values('tip', ascending=False) #сортировка данных по колонке tips в порядке убывания

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
170,50.81,10.00,Male,Yes,Sat,Dinner,3,16.94,Gregory Clark,5473850968388236,Sat1954
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139
141,34.30,6.70,Male,No,Thur,Lunch,6,5.72,Steven Carlson,3526515703718508,Thur1025
...,...,...,...,...,...,...,...,...,...,...,...
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
111,7.25,1.00,Female,No,Sat,Dinner,1,7.25,Terri Jones,3559221007826887,Sat4801
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455


In [ ]:
df.sort_values(['tip', 'size'])

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455
111,7.25,1.00,Female,No,Sat,Dinner,1,7.25,Terri Jones,3559221007826887,Sat4801
92,5.75,1.00,Female,Yes,Fri,Dinner,2,2.88,Leah Ramirez,3508911676966392,Fri3780
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
...,...,...,...,...,...,...,...,...,...,...,...
141,34.30,6.70,Male,No,Thur,Lunch,6,5.72,Steven Carlson,3526515703718508,Thur1025
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590


Сначала была произведена сортировка по колонке tip, но если есть одинаковые значения, сортируем по колонке size

In [ ]:
df['total_bill'].max() #выводим значение самого большого счета колонки total_bill

50.81

In [ ]:
df['total_bill'].idxmax() #вывели индекс строки с самым большим значением колонки total_bill

170

In [ ]:
df.iloc[170]

total_bill                     50.81
tip                             10.0
sex                             Male
smoker                           Yes
day                              Sat
time                          Dinner
size                               3
price_per_person               16.94
Payer Name             Gregory Clark
CC Number           5473850968388236
Payment ID                   Sat1954
Name: 170, dtype: object

In [ ]:
df['total_bill'].min() #выводим значение самого маленького счета колонки total_bill

3.07

In [ ]:
df['total_bill'].idxmin() #вывели индекс строки с самым маленьким значением колонки total_bill

67

In [ ]:
df.iloc[67]

total_bill                      3.07
tip                              1.0
sex                           Female
smoker                           Yes
day                              Sat
time                          Dinner
size                               1
price_per_person                3.07
Payer Name             Tiffany Brock
CC Number           4359488526995267
Payment ID                   Sat3455
Name: 67, dtype: object

In [ ]:
df.corr() #выводит корреляцию медлу колонками

<ipython-input-32-b20381c2a15d>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.corr() #выводит корреляцию медлу колонками


,total_bill,tip,size,price_per_person,CC Number
total_bill,1.000000,0.675734,0.598315,0.647554,0.104576
tip,0.675734,1.000000,0.489299,0.347405,0.110857
size,0.598315,0.489299,1.000000,-0.175359,-0.030239
price_per_person,0.647554,0.347405,-0.175359,1.000000,0.135240
CC Number,0.104576,0.110857,-0.030239,0.135240,1.000000


В последних версиях Pandas параметр для выбора только числовых колонок df.corr(numeric_only=True)

In [ ]:
df.head()

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
2,21.01,3.50,Male,No,Sun,Dinner,3,7.00,Travis Walters,6011812112971322,Sun4458
3,23.68,3.31,Male,No,Sun,Dinner,2,11.84,Nathaniel Harris,4676137647685994,Sun5260
4,24.59,3.61,Female,No,Sun,Dinner,4,6.15,Tonya Carter,4832732618637221,Sun2251


In [ ]:
df['sex'].value_counts() #посчитали количество строк с мужским и женским полом

Male      157
Female     87
Name: sex, dtype: int64

In [ ]:
df['day'].unique() #получаем массив с уникальными значениями колонки day

array(['Sun', 'Sat', 'Thur', 'Fri'], dtype=object)

In [ ]:
df['day'].nunique() #получаем количество уникальных значений колонки day

4

In [ ]:
df['day'].value_counts() #количество строк с каждым значением колонки day

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

Замена значений 2 способа:

In [ ]:
#1)
df['sex'].replace('Female', 'F') #заменили значения sex с Female на F

0         F
1      Male
2      Male
3      Male
4         F
       ... 
239    Male
240       F
241    Male
242    Male
243       F
Name: sex, Length: 244, dtype: object

In [ ]:
df['sex'].replace(['Female', 'Male'], ['F', 'M']) #заменили значения колонки секс с Female и Male на F и M

#удобен для замены одного или нескольких значений

0      F
1      M
2      M
3      M
4      F
      ..
239    M
240    F
241    M
242    M
243    F
Name: sex, Length: 244, dtype: object

In [ ]:
#2)
my_map = {'Female':'F', 'Male':'M'} #также заменили значения колонки секс с Female и Male на F и M
df['sex'].map(my_map)

#удобен для замены большого количества значений

0      F
1      M
2      M
3      M
4      F
      ..
239    M
240    F
241    M
242    M
243    F
Name: sex, Length: 244, dtype: object

In [ ]:
df.duplicated() #выводит False, если нет дубликатов и True, если они есть

0      False
1      False
2      False
3      False
4      False
       ...  
239    False
240    False
241    False
242    False
243    False
Length: 244, dtype: bool

In [ ]:
simple_df = pd.DataFrame([1,2,2,2], ['a', 'b', 'c', 'd'])

In [ ]:
simple_df.duplicated()

#для первой строки будет False, а дальше, если есть повтор, будет True

a    False
b    False
c     True
d     True
dtype: bool

In [ ]:
simple_df.drop_duplicates() #оставляет один экземляр строк и удаляет дубликаты

,0
a,1
b,2


In [ ]:
df['total_bill'].between(10,20, inclusive='both')

#выводит все значения колонки total_bill в диапазоне от 10 до 20. inclusive='both' - включая 20, inclusive='left' не включая 20
#в старых версиях pandas inclusive = True или inclusive = False

0       True
1       True
2      False
3      False
4      False
       ...  
239    False
240    False
241    False
242     True
243     True
Name: total_bill, Length: 244, dtype: bool

In [ ]:
df[df['total_bill'].between(10,20,inclusive='both')] #вывели строки только с данными, которые входят в диапазон

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
1,10.34,1.66,Male,No,Sun,Dinner,3,3.45,Douglas Tucker,4478071379779230,Sun4608
8,15.04,1.96,Male,No,Sun,Dinner,2,7.52,Joseph Mcdonald,3522866365840377,Sun6820
9,14.78,3.23,Male,No,Sun,Dinner,2,7.39,Jerome Abbott,3532124519049786,Sun3775
10,10.27,1.71,Male,No,Sun,Dinner,2,5.14,William Riley,566287581219,Sun2546
...,...,...,...,...,...,...,...,...,...,...,...
234,15.53,3.00,Male,Yes,Sat,Dinner,2,7.76,Tracy Douglas,4097938155941930,Sat7220
235,10.07,1.25,Male,No,Sat,Dinner,2,5.04,Sean Gonzalez,3534021246117605,Sat4615
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
242,17.82,1.75,Male,No,Sat,Dinner,2,8.91,Dennis Dixon,4375220550950,Sat17


In [ ]:
df.nlargest(10,'tip')

#вывели 10 строк с наибольшими значениями колонки tip по убыванию

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
170,50.81,10.00,Male,Yes,Sat,Dinner,3,16.94,Gregory Clark,5473850968388236,Sat1954
212,48.33,9.00,Male,No,Sat,Dinner,4,12.08,Alex Williamson,676218815212,Sat4590
23,39.42,7.58,Male,No,Sat,Dinner,4,9.86,Lance Peterson,3542584061609808,Sat239
59,48.27,6.73,Male,No,Sat,Dinner,4,12.07,Brian Ortiz,6596453823950595,Sat8139
141,34.30,6.70,Male,No,Thur,Lunch,6,5.72,Steven Carlson,3526515703718508,Thur1025
183,23.17,6.50,Male,Yes,Sun,Dinner,4,5.79,Dr. Michael James,4718501859162,Sun6059
214,28.17,6.50,Female,Yes,Sat,Dinner,3,9.39,Marissa Jackson,4922302538691962,Sat3374
47,32.40,6.00,Male,No,Sun,Dinner,4,8.10,James Barnes,3552002592874186,Sun9677
239,29.03,5.92,Male,No,Sat,Dinner,3,9.68,Michael Avila,5296068606052842,Sat2657
88,24.71,5.85,Male,No,Thur,Lunch,2,12.36,Roger Taylor,4410248629955,Thur9003


In [ ]:
df.nsmallest(10,'tip')

#вывели 10 строк с наименьшими значениями колонки tip по возрастанию

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
67,3.07,1.00,Female,Yes,Sat,Dinner,1,3.07,Tiffany Brock,4359488526995267,Sat3455
92,5.75,1.00,Female,Yes,Fri,Dinner,2,2.88,Leah Ramirez,3508911676966392,Fri3780
111,7.25,1.00,Female,No,Sat,Dinner,1,7.25,Terri Jones,3559221007826887,Sat4801
236,12.60,1.00,Male,Yes,Sat,Dinner,2,6.30,Matthew Myers,3543676378973965,Sat5032
0,16.99,1.01,Female,No,Sun,Dinner,2,8.49,Christy Cunningham,3560325168603410,Sun2959
215,12.90,1.10,Female,Yes,Sat,Dinner,2,6.45,Jessica Owen,4726904879471,Sat6983
237,32.83,1.17,Male,Yes,Sat,Dinner,2,16.42,Thomas Brown,4284722681265508,Sat2929
75,10.51,1.25,Male,No,Sat,Dinner,2,5.26,Kenneth Hayes,213142079731108,Sat5056
135,8.51,1.25,Female,No,Thur,Lunch,2,4.26,Rebecca Harris,4320272020376174,Thur6600
235,10.07,1.25,Male,No,Sat,Dinner,2,5.04,Sean Gonzalez,3534021246117605,Sat4615


In [ ]:
df.sample(5) #вывели 5 случайных строк

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
43,9.68,1.32,Male,No,Sun,Dinner,2,4.84,Christopher Spears,4387671121369212,Sun3279
135,8.51,1.25,Female,No,Thur,Lunch,2,4.26,Rebecca Harris,4320272020376174,Thur6600
56,38.01,3.00,Male,Yes,Sat,Dinner,4,9.50,James Christensen DDS,349793629453226,Sat8903
125,29.80,4.20,Female,No,Thur,Lunch,6,4.97,Angela Sanchez,503857080488,Thur3948
198,13.00,2.00,Female,Yes,Thur,Lunch,2,6.50,Katherine Bond,4926725945192,Thur437


In [ ]:
df.sample(frac = 0.1) #вывели случайные 10% всех строк

,total_bill,tip,sex,smoker,day,time,size,price_per_person,Payer Name,CC Number,Payment ID
228,13.28,2.72,Male,No,Sat,Dinner,2,6.64,Glenn Jones,502061651712,Sat2937
12,15.42,1.57,Male,No,Sun,Dinner,2,7.71,Chad Harrington,577040572932,Sun1300
38,18.69,2.31,Male,No,Sat,Dinner,3,6.23,Brandon Bradley,4427601595688633,Sat4056
81,16.66,3.40,Male,No,Thur,Lunch,2,8.33,William Martin,4550549048402707,Thur8232
126,8.52,1.48,Male,No,Thur,Lunch,2,4.26,Mario Bradshaw,4524404353861811,Thur6719
52,34.81,5.20,Female,No,Sun,Dinner,4,8.70,Emily Daniel,4291280793094374,Sun6165
200,18.71,4.00,Male,Yes,Thur,Lunch,3,6.24,Jason Conrad,4581233003487,Thur6048
217,11.59,1.50,Male,Yes,Sat,Dinner,2,5.80,Gary Orr,30324521283406,Sat8489
151,13.13,2.00,Male,No,Sun,Dinner,2,6.56,Jason Arnold,3571825125296106,Sun2127
177,14.48,2.00,Male,Yes,Sun,Dinner,2,7.24,John Dudley,4565183162071073,Sun6203
